<h1>Ditchley S2DS project August 2020 - Code Pipeline<h1>
    <h2>Team: Adam Hawken, Luca Lamoni, Elizabeth Nicholson, Robert Webster<h2>

In [1]:
# Import modules and set up working directory
import sys
import os
import time
import logging
import json
import csv
import threading
import queue
import asyncio 
import nest_asyncio
nest_asyncio.apply()
import twint
import pandas as pd

sys.path.insert(1, 'C:/Users/Luca/Aug20_Ditchley')

<h3>Section 1: Getting journalist twitter handles according to a keyword<h3>
    <h4>The journalist scraping is performed at the web address https://www.journalism.co.uk/prof/?chunk=0&cmd=default<h4>

In [2]:
from src.data import journalists as journos
keyword = 'cybersecurity'
journo_handles = journos.get_handles_by_keyword(keyword)
print(len(journo_handles))

3


<h3>Section 2. Loop over selected journalists handles and download their tweets<h3>
    <h4>Section 2.1: Download tweets using Twint<h4> 

In [3]:
from src.data import twint_tools as tt
# define keyword arguments
kwargs = {'date_range':('2020-08-01 00:00:00', None),
         'n_retries':5,
         'suppress':False}
# multi threading
tt.twint_in_queue(tt._search_tweets_by_user, 3, journo_handles, args=('../data/raw/'+keyword+'_',), kwargs=kwargs)

Attempt #1 to get tweets of @_lucyinghamAttempt #1 to get tweets of @JesscaHaworthAttempt #1 to get tweets of @Ad_Nauseum74




CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable


Results for @Ad_Nauseum74 saved to: ../data/raw/cybersecurity_tweets_Ad_Nauseum74.csv
Results for @JesscaHaworth saved to: ../data/raw/cybersecurity_tweets_JesscaHaworth.csv
Results for @_lucyingham saved to: ../data/raw/cybersecurity_tweets__lucyingham.csv


In [4]:
# Joined all the individual csv into one dataframe
cyber_test = tt.join_tweet_csv(journo_handles, keyword)
# Check
cyber_test.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1296054092234719233,1296054092234719233,1597838349000,2020-08-19,13:59:09,Romance Daylight Time,335773502,_lucyingham,Lucy Ingham,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '335773502', 'username': '_lucyin...",NaN,NaN,NaN,NaN
1,1295467814304849920,1295467814304849920,1597698570000,2020-08-17,23:09:30,Romance Daylight Time,335773502,_lucyingham,Lucy Ingham,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '335773502', 'username': '_lucyin...",NaN,NaN,NaN,NaN
2,1295466789443338247,1295466789443338247,1597698325000,2020-08-17,23:05:25,Romance Daylight Time,335773502,_lucyingham,Lucy Ingham,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '335773502', 'username': '_lucyin...",NaN,NaN,NaN,NaN
3,1295463883759648772,1295460920991391746,1597697633000,2020-08-17,22:53:53,Romance Daylight Time,335773502,_lucyingham,Lucy Ingham,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '335773502', 'username': '_lucyin...",NaN,NaN,NaN,NaN
4,1295412414725455877,1295412328826056704,1597685361000,2020-08-17,19:29:21,Romance Daylight Time,335773502,_lucyingham,Lucy Ingham,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '335773502', 'username': '_lucyin...",NaN,NaN,NaN,NaN


In [5]:
# Standardise the twint output 
from src.data import data_cleanup as dc
#Create the standardized template
test_twint = dc.standard_tweet_dataset_setup('test_twint')
test_twint
#fill the template
standard_tweet_twint = dc.fill_standard_tweet_dataset_with_twint(test_twint, cyber_test)
# Check
standard_tweet_twint.head()

,tweet_id,conversation_id,in_reply_to_status_id,reply_to,in_reply_to_user_id,in_reply_to_screen_name,user_id,screen_name,tweet_created_at,text,replies_count,retweets_count,likes_count,hashtags,retweet_text,topics
0,1296054092234719233,1296054092234719233,NaN,"[{'user_id': '335773502', 'username': '_lucyin...",NaN,NaN,335773502,_lucyingham,2020-08-19 13:59:09,My take on Facebook's decision to require its ...,0,1,2,[],NaN,NaN
1,1295467814304849920,1295467814304849920,NaN,"[{'user_id': '335773502', 'username': '_lucyin...",NaN,NaN,335773502,_lucyingham,2020-08-17 23:09:30,"Fellow journalists, please add your name to @j...",0,1,3,"['#journalistsforbelarus', '#journorequest']",NaN,NaN
2,1295466789443338247,1295466789443338247,NaN,"[{'user_id': '335773502', 'username': '_lucyin...",NaN,NaN,335773502,_lucyingham,2020-08-17 23:05:25,Just re-watched Pirates of Silicon Valley afte...,0,0,0,['#anthonymichaelhall'],NaN,NaN
3,1295463883759648772,1295460920991391746,NaN,"[{'user_id': '335773502', 'username': '_lucyin...",NaN,NaN,335773502,_lucyingham,2020-08-17 22:53:53,Same!,0,0,1,[],NaN,NaN
4,1295412414725455877,1295412328826056704,NaN,"[{'user_id': '335773502', 'username': '_lucyin...",NaN,NaN,335773502,_lucyingham,2020-08-17 19:29:21,Disclaimer: my javascript isn't perfect,0,0,1,[],NaN,NaN


In [6]:
# Save the dataframe
standard_tweet_twint.to_csv('../data/processed/' + keyword + '_standard_tweets_twint.csv',index=False)

<h4>Section 2.2: Extract mentions from Twint dataset<h4> 

In [7]:
# from the twint dataset, extract mentions based on tweet id and save in a separate csv
mentions_twint  = dc.twint_mentions_to_df(cyber_test)
# Check
mentions_twint.head()

,tweet_id,mentions
0,1295467814304849920,journalists4bel
1,1295463883759648772,filleboheme
2,1295463883759648772,terryandrob
3,1294963552902680578,jonnelledge
4,1293479566120570885,ukandeu


In [8]:
# Save the dataframe
mentions_twint.to_csv('../data/processed/' + keyword + '_mentions_twint.csv',index=False)

<h4>Section 2.3: Scrape list of friends for each journalist<h4> 

In [9]:
from src.data import twint_tools as tt
# define keyword arguments
kwargs = {#'date_range':('2020-08-01 00:00:00', None),
         'n_retries':5,
         'suppress':False}
# multi threading
tt.twint_in_queue(tt._get_friends, 3, journo_handles, args=('../data/raw/'+keyword+'_',), kwargs=kwargs)

Attempt #1 to get friends of @_lucyingham
Attempt #1 to get friends of @JesscaHaworth
Attempt #1 to get friends of @Ad_Nauseum74


CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.feed:Follow:IndexError


Attempt #2 to get friends of @JesscaHaworth


CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.feed:Follow:IndexError


Attempt #3 to get friends of @JesscaHaworth


...

Results for @Ad_Nauseum74 saved to: ../data/raw/cybersecurity_friends_Ad_Nauseum74.csv


CRITICAL:root:twint.feed:Follow:IndexError
CRITICAL:root:twint.feed:Follow:IndexError


Results for @_lucyingham saved to: ../data/raw/cybersecurity_friends__lucyingham.csv


CRITICAL:root:twint.feed:Follow:IndexError
CRITICAL:root:twint.feed:Follow:IndexError


Results for @JesscaHaworth saved to: ../data/raw/cybersecurity_friends_JesscaHaworth.csv


In [10]:
# Concatenate all the individual lists into one dataframe with journalist and its friends
friends_csv = tt.join_friends_csv(journo_handles,keyword) # this function has a bug, the first friend name is 'username'

@_lucyingham follows 512 users.
@JesscaHaworth follows 970 users.
@Ad_Nauseum74 follows 367 users.

Total number of handles pulled: 1849
Number of unique twitter handles: 1707

Zero following in list for users: []


In [11]:
# Save the dataframe
friends_csv.to_csv('../data/raw/'+keyword+'_journalist_friends.csv', index=False)

<h4>Section 2b: Download tweets using API<h4>

<h3>Section 3. Loop over selected journalists handles and download their lists of friends<h3>